# Welcome to the Biotek Package

The Biotek package of `murraylab_tools` is designed to make your life easier when analyzing Biotek time series data. Specifically, murraylab_tools.Biotek converts Biotek data to tidy format for easy analysis with Pandas and easy plotting with Seaborn. The package also contains a few convenience functions for some of the things you're likely to do with TX-TL time series data, currently including:
* Background subtraction
* Endpoint summarization.

### Data Tidying

Converting from Biotek output to a tidy, Pandas-readable format is simple:

In [ ]:
import murraylab_tools.biotek as mt_biotek
import os

# Note that the input file must be from excel output of a Biotek experiment,
# saved in CSV format. 
data_filename = os.path.join("biotek_examples", "RFP_GFP_traces.csv")
mt_biotek.tidy_biotek_data(data_filename)

A tidified version of the data will be created with the same name and location as the original time trace file, with "_tidy" appended to the end of the name (pre-suffix).

Usually you will also want access to some meta-data on your experiment -- most importantly, what plasmids were put in each well, in what concentration. You can add this metadata in the form of a supplementary spreadsheet. The first column of the supplementary spreadsheet is assumed to contain a well number (e.g., "D4" or "A08"). Every other column contains some kind of metadata keyed to that well number, with a name of that metadata given in a header row. You can write that supplementary data file yourself in Excel or notepad. 

Here is an example of how to write a supplementary file programmatically. This experiment contains three replicated 2D titrations of a GFP plasmid on one axis (at concentrations of 0.25, 0.5, 1, and 2 nM) and an RFP plasmid on the other axis (at concentrations of 0.5, 1, 2, and 4 nM).

In [ ]:
import csv, string

rfp_concs = [0.44, 0.88, 2.20, 3.97]
gfp_concs = [0.24, 0.49, 0.98, 2.01]
replicates = [1,2,3]

supplementary_filename = os.path.join("biotek_examples", "RFP_GFP_supplementary.csv")
with open(supplementary_filename, 'w') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['Well', 'RFP Plasmid (nM)', 'GFP Plasmid (nM)', 'Replicate'])
    for row in range(4):
        for col in range(4):
            well_names = [string.ascii_uppercase[row] + "%d" % (col+6),
                          string.ascii_uppercase[row] + "%d" % (col+11),
                          string.ascii_uppercase[row+6] + "%d" % (col+1)]
            for rep in replicates:
                writer.writerow([well_names[rep-1], rfp_concs[row], gfp_concs[col], 
                                  rep])
    # Also include metadata for three negative control wells.
    writer.writerow(["E10", 0, 0, 1])
    writer.writerow(["E15", 0, 0, 2])
    writer.writerow(["K5", 0, 0, 3])
                

and to use that supplementary data:

In [ ]:
mt_biotek.tidy_biotek_data(data_filename, supplementary_filename)

Now we can easily read our Biotek data using Pandas:

In [ ]:
import pandas as pd

tidy_filename = os.path.join("biotek_examples", "RFP_GFP_traces_tidy.csv")
df = pd.read_csv(tidy_filename)

Let's peek at the data in its tidy format:

In [ ]:
df.head()

As you can see, each row of tidy data describes one well's read value for a single channel and gain at a single time, along with some metadata about that well.

Now we can use Seaborn to start plotting data quickly and (relatively) easily. In this case, let's look at time traces of all three replicates of each combination of plasmids, first for GFP and then for RFP:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
rc={'lines.linewidth': 2, 'axes.labelsize': 14, 'axes.titlesize': 14}
sns.set(rc=rc)
%matplotlib inline

gfp_df = df[(df.Gain == 100) & (df.Channel == "GFP")]
rfp_df = df[(df.Gain == 100) & (df.Channel == "RFP")]
grid = sns.FacetGrid(gfp_df, col = "GFP Plasmid (nM)", row = "RFP Plasmid (nM)", 
                     hue = "Replicate", margin_titles = True)
grid.map(plt.plot, "Time (hr)", "uM")
grid.fig.tight_layout(w_pad=1)
plt.show()

plt.clf()
grid = sns.FacetGrid(rfp_df, col = "GFP Plasmid (nM)", row = "RFP Plasmid (nM)", 
                     hue = "Replicate", margin_titles = True)
grid.map(plt.plot, "Time (hr)", "uM")
grid.fig.tight_layout(w_pad=1)
plt.show()

### Automatic Unit Conversion

Note in the example above that the absolute concentrations of fluorescent proteins were automatically calculated in µM. The murraylab_tools.biotek package has a small database of known calibrations for our lab's Biotek's. It currently knows how to calibrate with GFP, Citrine, RFP, CFP, Venus, and Cherry, each at gains 61 and 100. The plate ID is pulled from the Biotek's output file. Fluorescent protein identities are pulled from channel names. If the name of a channel is sufficiently similar to one of the known fluorescent proteins, you will get automatic unit conversion. Here "sufficiently similar" means "contains as a substring". If you use a channel name with more than one known fluorescent protein name in it, you're not guaranteed any particular result, so don't do that.  

If you're feeling particularly meddlesome (or you want to add new calibration data to the package -- if so, please commit your changes to the repository!), you can see the calibration data near the top of the biotek.py file in the biotek subpackage of murraylab_tools. Calibration data is stored as a doubly-nested dictionary of the form

`calibration_data[channel][biotek_id][gain] = AFU/µM`.

If you don't use a channel containing the name of a calibrated substance, or you use a gain that hasn't been calibrated, then the µM concentration will be calculated as -1. 

# Convenience Functions

### Background Subtraction

Given a list of negative control wells, you can use the murraylab_tools.biotek package to subtract average background from your time series data:

In [ ]:
negative_control_wells = ["E10", "E15", "K5"]
corrected_df = mt_biotek.background_subtract(df, negative_control_wells)
corrected_df.head()

`background_subtract` returns a dataframe of corrected data, which can be analyzed just like the dataframes returned in the example above. 

### Endpoint Averaging

If you know your fluorescence data plateaus near the end of the run, you may want to quickly find the endpoint fluorescence of each well, averaged over the last few time points. You can quickly do this with the `endpoint_averages` function of the murraylab_tools.biotek package.

In [ ]:
endpoint_df = mt_biotek.endpoint_averages(corrected_df)
endpoint_df.head()

Note that every numeric field has been averaged over the last five data points, so don't take the time column too seriously here. 

Now we can quickly plot endpoint averages for the two color channels:

In [ ]:
green_endpoints = endpoint_df[(endpoint_df.Channel == "GFP") & (endpoint_df.Gain == 61)]
sns.lmplot(data = green_endpoints, x = 'GFP Plasmid (nM)', 
              y = 'uM', hue = 'RFP Plasmid (nM)', fit_reg = False,
              x_jitter = 0.05)

In [ ]:
red_endpoints = endpoint_df[(endpoint_df.Channel == "RFP") & (endpoint_df.Gain == 100)]
sns.lmplot(data = red_endpoints, x = 'RFP Plasmid (nM)', 
              y = 'uM', hue = 'GFP Plasmid (nM)', fit_reg = False,
              x_jitter = 0.05)

### Manual Unit Conversion

If you want to manually convert an AFU readout to absolute concentration, you can 